In [9]:
import fitz
import chromadb
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from docx import Document
from langchain_ollama import OllamaLLM

In [12]:
def get_pdf(file):
    text = ""
    if file.endswith(".pdf"):
        document = fitz.open(file)

        for page in document:
            text+=page.get_text()
    elif file.endswith(".DOCX"):
        document = Document(file)
        for page in document.paragraphs:
            text+=page.text+'\n'
    return text

In [13]:
print(get_pdf('PGP-2425-PR-2 (REL).DOCX'))


Planeamento e Gestão de Projeto
Ano Letivo 2024/2025
Tema do Projeto
Relatório da Etapa 2
Grupo YY
Nome 1	99999
Nome 2	99999
Nome 3	99999
Nome 4	99999
Nome 5	99999
Nome 6	99999

Índice
Introdução	3
1. Recursos	3
1.1. Pessoas da equipa	3
1.2. Ferramentas de desenvolvimento	3
1.3. Software para o sistema	3
2. Estimativas	4
2.1. Esforço disponível	4
2.2. Dados históricos	4
2.3. Estimativas baseadas em linhas de código	4
2.4. Estimativas baseadas no modelo COCOMO	4
3. Processo de desenvolvimento de software	5
4. Organização da equipa	5
5. Planeamento do projeto	5
5.1. Work packages	5
5.2. Calendarização	6
6. Gestão de riscos	6
6.1. Lista de riscos	6
6.2. Tabela de riscos	6
6.3. Plano RMMM	6
Conclusão	6
Bibliografia	7
Apêndice	8


Introdução
Descrição do contexto, problema a resolver e sua importância, objetivos e mais valias do projeto, e síntese dos aspetos cobertos no relatório. Deve ser citado o enunciado do projeto, pois é aí que estão os requisitos de informação, funcionais, e não fu

In [8]:
def get_pdf(file):
    document = fitz.open(file)
    text=""
    for page in document:
        text+=page.get_text()
    return text

In [3]:
print(get_pdf('test.pdf'))

test
aici testez filurile
test
1



In [9]:
def chunking(text):
    text_splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder(encoding_name="cl100k_base",chunk_size=500,chunk_overlap=50)
    texts=text_splitter.split_text(text)
    return texts

In [29]:
text=get_pdf('T04_Camada_Rede.pdf')
print(chunking(text))

['T04- Camada rede\nSubjects\nRC\nServiços da camada de rede\nTransporte de datagramas entre computadores\nOs protocolos desta camada correm nos nós terminais e nos routers \nOs routers examinam os cabeçalhos IP de todos os datagramas\nDuas funções chave\nRouting → Determine o caminho origem-destino , Ação global\nForwarding → Mover pacotes que chega no router para a saída apropriada; \nAção local no router\nAnalogia: \nO routing e planeamento de uma viagem\nO forwarding e a passar um cruzamento \nRouting vs forwarding \nalgoritmo de routing → Determina a rota através da rede\ntabela forwarding local → Determina como o pacote deve ser encaminhado neste \nrouter \nPlano de dados e Plano de controlo\nPlano de dados:\nPlano de controlo:\nT04- Camada rede\n1\nfunção local em cada router\ndetermina o caminho de \ndatagrama e a saida\nencaminhamento\nlogica global\nrouting e definido aqui\nduas abordagens:\ntradicional : algoritmos de \nrouting implementados nos \nrouters\nredes definidas po

In [47]:
model = SentenceTransformer('all-MiniLM-L6-v2')
def transformer(chunk):

    embeddings = model.encode(chunk).tolist()
    return embeddings

In [44]:
def vector_db(chunks,chunk_transformed):
    client=chromadb.Client()
    try:
        collection=client.create_collection('my_collection')
        print("created collection")
    except Exception as e:
        collection=client.get_collection('my_collection')

    for i,embedding in enumerate(chunk_transformed):
        collection.add(
            documents=[chunks[i]],
            embeddings=[embedding],
            ids=[str(i)]
        )

    return collection

In [16]:
text=get_pdf('T04_Camada_Rede.pdf')
chunks=chunking(text)
chunks_transformed=transformer(chunks)
vector=vector_db(chunks,chunks_transformed)

print(vector)

Collection(name=my_collection)


In [19]:
text = get_pdf("T04_Camada_Rede.pdf")
print(f"Text extras (primii 300 caractere): {text[:300]}...")
print("---------------------------")
chunks = chunking(text)
print(f"Număr chunk-uri: {len(chunks)}")
print("---------------------------")
print(f"Primul chunk: {chunks[0]}")

embeddings = transformer(chunks)
print("---------------------------")
print(f"Dimensiune embedding primul chunk: {len(embeddings[0])}")
print("---------------------------")

collection = vector_db(chunks, embeddings)
print(f"Număr documente în colecție: {len(collection.get(include=["documents"])['documents'])}")

Text extras (primii 300 caractere): T04- Camada rede
Subjects
RC
Serviços da camada de rede
Transporte de datagramas entre computadores
Os protocolos desta camada correm nos nós terminais e nos routers 
Os routers examinam os cabeçalhos IP de todos os datagramas
Duas funções chave
Routing → Determine o caminho origem-destino , Ação gl...
---------------------------
Număr chunk-uri: 9
---------------------------
Primul chunk: T04- Camada rede
Subjects
RC
Serviços da camada de rede
Transporte de datagramas entre computadores
Os protocolos desta camada correm nos nós terminais e nos routers 
Os routers examinam os cabeçalhos IP de todos os datagramas
Duas funções chave
Routing → Determine o caminho origem-destino , Ação global
Forwarding → Mover pacotes que chega no router para a saída apropriada; 
Ação local no router
Analogia: 
O routing e planeamento de uma viagem
O forwarding e a passar um cruzamento 
Routing vs forwarding 
algoritmo de routing → Determina a rota através da rede
tabel

In [36]:
llm = OllamaLLM(model="llama3.2")
def query_vector_db(collection,query,top_k=3):
    query_embeddings = model.encode(query).tolist()
    result=collection.query(query_embeddings=[query_embeddings],n_results=top_k)
    relevant_text=result['documents'][0]
    context="\n".join(relevant_text)
    prompt=f"Answer the following question with this material:\n{context}\n Question: {query}"
    return llm(prompt)

In [48]:
if __name__ == "__main__":
    file_path = "T04_Camada_Rede.pdf"
    text = get_pdf(file_path)
    chunks = chunking(text)
    embeddings = transformer(chunks)
    collection = vector_db(chunks, embeddings)

    intrebare = "Care este subiectul principal al documentului?"
    raspuns = query_vector_db(collection, intrebare)
    print("Răspuns:", raspuns)

Răspuns: Subiectul principal al documentului este despre Networking și rețele, mai precis despre mecanismele de trimitere a datelor într-o rețea (multicast) și despre diferite soluții pentru optimizarea transferului de date. Documentul abordează teme como:

- Pruning, o curățare a routerelor care nu au destinatari
- Arbores partilhate pentru a distribui datelor în mod eficient între membrii unei rețele
- Center-Based Tree, o metoda de trimitere a datelor multicast prin utilizarea unui punct central (rendezvous point)
- Problema de NAT și cum se poate rezolva
- Algoritmi de routing, inclusiv clasificarea lor în funcție de informații pe care le conțin și de modalitățile în care distribuiesc date intr-un rețea.
